Beaching notebook

This notebook is going to explore the beaching behaviour of particles in the 2D coral flow model. We want to know what the flowfields, particle position and interpolation look like the moment before particles are beached. We can use any ParticleSet output.

In [65]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import xarray as xr
import cmocean
import matplotlib.animation as animation

name = 'output-corals-regridded'
fb = 'forward' #variable to determine whether the flowfields are analysed 'forward' or 'backward' in time
runtime=timedelta(seconds=10)                            # total time to execute the particleset
outputdt = timedelta(seconds=0.01)
filename = name+'.nc'
dfilename = name+fb+str(runtime.seconds)+'.nc'

In [2]:
data = xr.open_dataset(dfilename)

flowdata = xr.open_dataset(filename)

Now that the data and flowfields are in the xarrray, we want to select only the beached particles

In [32]:
bdata = data.where(data['finaldistance'][:,-1]==0,drop=True)

Next we want to select only one trajectory

In [33]:
traj = 5
one = bdata.isel(traj=traj)
onetraj = one.where(one['finaldistance']==0,drop=True)

In [34]:
divider = 1000*outputdt.microseconds
tb = round(float(onetraj.isel(obs=1)['time'].values)/divider)
TB = round(float(onetraj.isel(obs=1)['time'].values)/1000000000)
print(TB)
print(tb)

2
164


In [11]:
x,y = np.meshgrid(flowdata['X'],flowdata['Y'])
um = np.ma.masked_invalid(flowdata['U'][0,:,:])                    # retrieve mask from flowfield to take out points over coral objects

lons = np.ma.masked_array(x,mask=um.mask)             # mask points in meshgrid
lons = lons.flatten()
ds = np.ma.masked_array(y,mask=um.mask)                 # mask points in meshgrid
ds = ds.flatten()

usm = np.invert(um.mask)
lonm = np.ma.masked_array(x,mask=usm)             # mask points in meshgrid
lonm = lonm.flatten()
dm = np.ma.masked_array(y,mask=usm)                 # mask points in meshgrid
dm = dm.flatten()

In [36]:
%matplotlib qt
fig = plt.figure(figsize=(13,10))
ax = plt.axes()
ax.plot(one['lon'].isel(obs=slice(tb-40,tb+1)),-one['z'].isel(obs=slice(tb-40,tb+1)),c='r',marker='o',label='particle trajectory')
ax.scatter(lons,ds,s=7,c='c',label='flow gridpoint')
ax.scatter(lonm,dm,s=15,c='k',label='coral gridpoint')
C = np.hypot(flowdata['U'].isel(T=TB),flowdata['V'].isel(T=TB))
ax.quiver(flowdata['X'],flowdata['Y'],flowdata['U'].isel(T=TB),flowdata['V'].isel(T=TB),C,angles='xy',scale=50)
plt.ylim(-one['z'].isel(obs=tb)-0.05,-one['z'].isel(obs=tb)+0.05)
plt.xlim(one['lon'].isel(obs=tb)-0.05,one['lon'].isel(obs=tb)+0.05)
plt.ylabel('Height [m]')
plt.xlabel('Horizontal distance [m]')
plt.title('Beaching trajectory '+str(traj))
plt.legend()
plt.savefig('Figures/beachingTrajectory'+str(traj))

In [9]:
ct = np.zeros(len(data['finaldistance'][0]))
ct[-1] = np.count_nonzero(data['finaldistance'][:,-1]==0)
beac = np.zeros((int(ct[-1])),dtype=np.int64)
# beac[:,1] = np.nonzero(data['finaldistance'][:,-1]==0)[0]
bn = data['finaldistance'].where(data['finaldistance'][:,-1]==0,drop=True).values
for i in range(len(bn)):
    cn = 0
    for j in np.arange(len(bn[i])-2,-1,-1):
        if bn[i,j] == 0:
            cn += 1
        else:
            break
    beac[i] = 44 - cn/10
for i in np.arange(len(data['finaldistance'][0])-2,-1,-1): # each timestep
    ct[i] = np.count_nonzero(data['finaldistance'][:,i]==0) #count how many values are zero 

In [10]:
plt.hist(beac)
plt.title('Particles beached per timebin')
plt.ylabel('number of particles')
plt.xlabel('time beached [s]')
plt.ylim(bottom=0)

(0, 768.6)

In [112]:
fig = plt.figure(figsize=(18,2))
ax = plt.axes()
ax.set_facecolor('k')
cf = ax.contourf(flowdata['X'],flowdata['Y'],flowdata['U'][8,:,:],np.linspace(-10,-9,21),extend='both',cmap=cmocean.cm.dense_r)
sc = ax.scatter(bdata['lon'][:,0], -bdata['z'][:,0], c=bdata['z'][:,0],s=10, marker="o",cmap='Spectral_r')
topleft = 0.05
if fb == 'backward':
    topleft = 0.8
time_text = ax.text(topleft, 0.95,'',horizontalalignment='left',verticalalignment='top', transform=ax.transAxes)
cbar = fig.colorbar(sc,label='Initial depth [m]')
cbar.ax.invert_yaxis()

def animate(i,fig,sc):
    sc.set_offsets(np.c_[bdata['lon'][:,i], -bdata['z'][:,i]])
    ts = i*outputdt.microseconds/1000000
    if fb == 'b':
        ts = ts*-1
    time_text.set_text('time = %.1f seconds' % ts)
    return sc, time_text,

ax.set_xlabel("x [m]")
ax.set_ylabel("z (m)")
ax.set_xlim(-0.5,8.5)
ax.set_ylim(-0.49,0.5)
anim = animation.FuncAnimation(fig, animate, fargs = (fig, sc),
                               frames = len(bdata['lon'][0]), blit=True)
anim.save('Figures/beaching.mp4')
plt.show()

Now we are going to look at the mass fluxes in each cell between 4 gridpoints to see if mass is conserved

In [47]:
massflux = np.zeros((len(flowdata['T']),len(flowdata['X'])-1,len(flowdata['Y'])-1))
dy = float(flowdata['Y'][0]-flowdata['Y'][1])
dx = float(flowdata['X'][1]-flowdata['X'][0])

# for i in range(len(flowdata['X'])-1):
#     for j in range(len(flowdata['Y'])-1):
#         massflux[:,i,j] =  np.sum([np.sum(flowdata['U'][:,j:j+1,i])/2*float(dy),
#                            - np.sum(flowdata['U'][:,j:j+1,i+1])/2*float(dy),
#                            - np.sum(flowdata['V'][:,j,i:i+1])/2*float(dx),
#                            np.sum(flowdata['V'][:,j+1,i:i+1])/2*float(dx)],axis=0)

In [ ]:
massflux = np.zeros((len(flowdata['X'])-1,len(flowdata['Y'])-1))
for i in range(len(flowdata['X'])-1):
    
    for j in range(len(flowdata['Y'])-1):
        massflux[i,j] = np.sum([np.sum(flowdata['U'][0,j:j+1,i])/2*float(dy),
                           - np.sum(flowdata['U'][0,j:j+1,i+1])/2*float(dy),
                           - np.sum(flowdata['V'][0,j,i:i+1])/2*float(dx),
                           np.sum(flowdata['V'][0,j+1,i:i+1])/2*float(dx)],axis=0)
    print(i)

In [44]:
np.sum(flowdata['V'][0,:,i:i+1],axis=1)/2*float(dx)

<xarray.DataArray 'V' (Y: 100)>
array([ 0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00, -5.e-05,
       -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05,
       -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05,
       -5.e-05, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04,
       -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04,
       -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04,
       -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04,
       -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04,
       -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04,
       -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04, -1.e-04,
       -1.e-04, -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05,
       -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05, -5.e-05,
        0.e+00,  0.e

In [51]:
print(V.shape)
print(U.shape)

(899, 100)
(900, 99)


In [63]:
V = np.zeros((len(flowdata['T']),len(flowdata['X'])-1,len(flowdata['Y'])))
U = np.zeros((len(flowdata['T']),len(flowdata['X']),len(flowdata['Y'])-1))
massflux = np.zeros((len(flowdata['T']),len(flowdata['X'])-1,len(flowdata['Y'])-1))

for i in range(len(flowdata['X'])-1):
    V[:,i,:] = np.sum(flowdata['V'][:,:,i:i+1],axis=2)/2*dx
print('done')
for j in range(len(flowdata['Y'])-1):
    U[:,:,j] = np.sum(flowdata['U'][:,j:j+1,:],axis=1)/2*dy
print('done again')

for i in range(len(flowdata['X'])-1):
    for j in range(len(flowdata['Y'])-1):
        massflux[:,i,j] = np.sum([U[:,i,j],-U[:,i+1,j],-V[:,i,j],V[:,i,j+1]],axis=0)
    print(i)



done
done again
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
27

In [57]:
np.save('massfluxest0',massflux,allow_pickle=True)

In [94]:
divergence = -massflux/dy/dx

In [72]:
%matplotlib qt
fig = plt.figure(figsize=(18,5))
ax = plt.axes()
# ax.plot(one['lon'].isel(obs=slice(tb-40,tb+1)),-one['z'].isel(obs=slice(tb-40,tb+1)),c='r',marker='o',label='particle trajectory')
# pc = ax.pcolormesh(x,y,divergence[TB].transpose(),vmin=-0.01,vmax=0.01,cmap=cmocean.cm.balance)

cbar = fig.colorbar(pc,label='Massflux [$m^2/s$]')
# ax.scatter(lons,ds,s=1,c='c',label='flow gridpoint')
# ax.scatter(lonm,dm,s=1,c='k',label='coral gridpoint')
ax.set_xlabel("x [m]")
ax.set_ylabel("z (m)")

Text(0, 0.5, 'z (m)')

In [111]:
%matplotlib qt
fig = plt.figure(figsize=(18,4))
ax = plt.axes()
# ax.plot(one['lon'].isel(obs=slice(tb-40,tb+1)),-one['z'].isel(obs=slice(tb-40,tb+1)),c='r',marker='o',label='particle trajectory')
pc = ax.pcolormesh(x,y,divergence[0].transpose(),vmin=-50,vmax=50,cmap=cmocean.cm.balance)
cbar = fig.colorbar(pc,label='Divergence [$1/s$]')
time_text = ax.text(0.05, 0.95,'time',horizontalalignment='left',verticalalignment='top', transform=ax.transAxes)
ax.set_xlabel("x [m]")
ax.set_ylabel("z (m)")
plt.title('Divergence of the velocity field [1/s]')
def animate(i):
    pc.set_array(divergence[i].transpose().flatten())
    ts = i
    time_text.set_text('time = %.1f seconds' % ts)
#     return time_text,

anim = animation.FuncAnimation(fig, animate,interval=1000, frames = len(flowdata['T']))

In [107]:
%matplotlib qt
fig = plt.figure(figsize=(18,4))
ax = plt.axes()
# ax.plot(one['lon'].isel(obs=slice(tb-40,tb+1)),-one['z'].isel(obs=slice(tb-40,tb+1)),c='r',marker='o',label='particle trajectory')
pc = ax.pcolormesh(x,y,massflux[0].transpose(),vmin=-np.max(abs(massflux)),vmax=np.max(abs(massflux)),cmap=cmocean.cm.balance)
cbar = fig.colorbar(pc,label='Flux [$m^2/s$]')
time_text = ax.text(0.05, 0.95,'time',horizontalalignment='left',verticalalignment='top', transform=ax.transAxes)
ax.set_xlabel("x [m]")
ax.set_ylabel("z (m)")
plt.title('Cumulative flux into gridcells [$m^2/s$]')
def animate(i):
    pc.set_array(massflux[i].transpose().flatten())
    ts = i
    time_text.set_text('time = %.1f seconds' % ts)
#     return time_text,

anim = animation.FuncAnimation(fig, animate,interval=1000, frames = len(flowdata['T']))